In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
import utils.risktool as risktool
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

#restore saved variables
%store -r summary_M2_KY_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
x = data.loc[:,:'current_violence20'].drop(['arnold_fta_raw', 'arnold_nca_raw', 'arnold_nvca_raw'], axis=1)
y = data['misdemeanor_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         seed=816)

#### EBM
estimators = [40]
depth = [2]
learning_rate = [0.3]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate=learning_rate,
                       depth=depth,
                       estimators=estimators,
                       seed =816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning:

div

In [4]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.7200825212070562, 0.01045183753113097)

In [6]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.7258293248451217, 0.0034441240350525427)

### Lasso Stumps

In [7]:
## load stumps & original data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on='person_id')

## split X & Y
X_stumps = data.loc[:,:'current_violence201'].copy()
Y_stumps = data['misdemeanor_two_year'].values.copy()
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test stumps data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_train_stumps.csv").sort_values('person_id')
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_test_stumps.csv").sort_values('person_id')
X_train_stumps = train_stumps.loc[:,:'current_violence201'].copy()
X_test_stumps = test_stumps.loc[:,:'current_violence201'].copy()
Y_train_stumps = train_stumps['misdemeanor_two_year'].values.copy()
Y_test_stumps = test_stumps['misdemeanor_two_year'].values.copy()
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

##### Single Stump Model

In [10]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.002, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

30

In [11]:
single_stump_model['features']

['age_at_current_charge24',
 'age_at_current_charge27',
 'age_at_current_charge35',
 'age_at_current_charge39',
 'age_at_current_charge43',
 'age_at_current_charge47',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_violence1',
 'p_felony1',
 'p_felony2',
 'p_felony3',
 'p_felony4',
 'p_misdemeanor2',
 'p_misdemeanor3',
 'p_misdemeanor4',
 'p_assault1',
 'p_traffic1',
 'p_drug1',
 'ADE1',
 'p_fta_two_year1',
 'p_fta_two_year2',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'six_month1',
 'one_year1']

#### Nested Cross Validation

In [12]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.001, 0.002]}, 
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars



In [13]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}],
 0.7209581901000105,
 0.0004233608620234586)

### RiskSLIM

In [26]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_stumps.csv").sort_values('person_id')

## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.0008,  
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

12

In [27]:
### Subset features
selected_features = single_stump_model['features']
if 'sex1' in selected_features:
    selected_features = ['misdemeanor_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['misdemeanor_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on='person_id')
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [28]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator = indicator,
                                       y_label='misdemeanor_two_year', 
                                       max_coef=5, 
                                       max_coef_number=5, 
                                       max_runtime=1000,
                                       max_offset=100,
                                       c=1e-6, 
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
02/12/20 @ 03:50 PM | 173 rows in lookup table
02/12/20 @ 03:50 PM | ------------------------------------------------------------
02/12/20 @ 03:50 PM | runnning initialization procedure
02/12/20 @ 03:50 PM | ------------------------------------------------------------
02/12/20 @ 03:50 PM | CPA produced 2 cuts
02/12/20 @ 03:50 PM | running naive rounding on 4 solutions
02/12/20 @ 03:50 PM | best objective value: 0.6931
02/12/20 @ 03:50 PM | rounding produced 4 integer solutions
02/12/20 @ 03:50 PM | best objective value is 0.6931
02/12/20 @ 03:50 PM | running sequential rounding on 4 solutions
02/12/20 @ 03:50 PM | best objective value: 0.6931
02/12/20 @ 03:50 PM | sequential rounding produced 0 integer solutions
02/12/20 @ 03:50 PM | polishing 4 solutions
02/12/20 @ 03:50 PM | best objective value: 0.6931
02/12/20 @ 03:50 PM | polishing produced 4 integer solutions
02/12/20 @ 03:50 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 03:50 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4253.
Tried aggregator 1 time.
Reduced MIP has 26 rows, 28 columns, and 75 nonzeros.
Reduced MIP has 12 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 03:50 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4258.
Tried aggregator 1 time.
Reduced MIP has 26 rows, 28 columns, and 75 nonzeros.
Reduced MIP has 12 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 03:50 PM | adding 236 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4250.
Tried aggregator 1 time.
Reduced MIP has 26 rows, 28 columns, and 75 nonzeros.
Reduced MIP has 12 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 03:50 PM | adding 241 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4255.
Tried aggregator 1 time.
Reduced MIP has 26 rows, 28 columns, and 75 nonzeros.
Reduced MIP has 12 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 03:54 PM | adding 246 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4244.
Tried aggregator 1 time.
Reduced MIP has 26 rows, 28 columns, and 75 nonzeros.
Reduced MIP has 12 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 03:54 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4243.
Tried aggregator 1 time.
Reduced MIP has 26 rows, 28 columns, and 75 nonzeros.
Reduced MIP has 12 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 03:54 PM | adding 248 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4248.
Tried aggregator 1 time.
Reduced MIP has 26 rows, 28 columns, and 75 nonzeros.
Reduced MIP has 12 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 03:54 PM | adding 241 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4246.
Tried aggregator 1 time.
Reduced MIP has 26 rows, 28 columns, and 75 nonzeros.
Reduced MIP has 12 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars



In [29]:
np.mean(riskslim_summary['validation_auc']), np.mean(riskslim_summary['test_auc'])

(0.6770118270349189, 0.6767894847709793)

#### Single RiskSLIM Model

In [30]:
selected_features = ["misdemeanor_two_year"] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [31]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'misdemeanor_two_year',
    'sample_weights': sample_weights
}

In [32]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-6, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
02/12/20 @ 04:01 PM | 173 rows in lookup table
02/12/20 @ 04:01 PM | ------------------------------------------------------------
02/12/20 @ 04:01 PM | runnning initialization procedure
02/12/20 @ 04:01 PM | ------------------------------------------------------------
02/12/20 @ 04:01 PM | CPA produced 2 cuts
02/12/20 @ 04:01 PM | running naive rounding on 4 solutions
02/12/20 @ 04:01 PM | best objective value: 0.6931
02/12/20 @ 04:01 PM | rounding produced 4 integer solutions
02/12/20 @ 04:01 PM | best objective value is 0.6931
02/12/20 @ 04:01 PM | running sequential rounding on 4 solutions
02/12/20 @ 04:01 PM | best objective value: 0.6931
02/12/20 @ 04:01 PM | sequential rounding produced 0 integer solutions
02/12/20 @ 04:01 PM | polishing 4 solutions
02/12/20 @ 04:01 PM | best objective value: 0.6931
02/12/20 @ 04:01 PM | polishing produced 4 integer solutions
02/12/20 @ 04:01 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 04:01 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4250.
Tried aggregator 1 time.
Reduced MIP has 26 rows, 28 columns, and 75 nonzeros.
Reduced MIP has 12 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [33]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [34]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.6783658304195097

In [35]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.6750965346168157

### Arnold PSA

In [36]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    data = data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    data[score] = data[score + "_raw"]
    data = data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        data[score] = data[score].astype(int)
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['misdemeanor_two_year'].values

In [37]:
arnold = risktool.risktool(X_arnold, Y_arnold, label='arnold_nca')

In [38]:
np.mean(arnold['auc'])

0.6860566178808941

#### Single Arnold PSA

In [39]:
### load data
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_test.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    test_data = test_data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    test_data[score] = test_data[score + "_raw"]
    test_data = test_data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        test_data[score] = test_data[score].astype(int)

In [40]:
X = test_data['arnold_nca'].values
Y = test_data['misdemeanor_two_year'].values
roc_auc_score(Y, X)

0.6819149971772178

### Results

In [41]:
#### save results
summary_M2_KY_interpret = {"cart": cart_summary,
                           "ebm": ebm_summary, 
                           'stumps': stump_summary, 
                           'riskslim': riskslim_summary, 
                           'arnold': arnold}
%store summary_M2_KY_interpret

Stored 'summary_M2_KY_interpret' (dict)


In [42]:
results = [["cart", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["ebm", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['riskslim', np.mean(riskslim_summary['test_auc'])],
           ['arnold', np.mean(arnold['auc'])]]
results

[['cart', 0.7200825212070562, 0.01045183753113097],
 ['ebm', 0.7258293248451217, 0.0034441240350525427],
 ['stumps', 0.7209581901000105, 0.0004233608620234586],
 ['riskslim', 0.6767894847709793],
 ['arnold', 0.6860566178808941]]

In [43]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc'])]

In [44]:
path = "./kentucky/logs/interpretable/"
results = [["Misdemeanor", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold['auc']), 3)) + " (" + np.str(round(np.std(arnold['auc']),3)) + ")"]]
with open(path + 'KY-two-year-interpretable-summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### Save Fairness Matrix

In [45]:
matrix = ['confusion_matrix_stats', 'calibration_stats', 'race_auc', 'condition_pn', 'no_condition_pn']
name = ['confusion', 'calibration', 'race-auc', 'condition-pn', 'no-condition-pn']

In [46]:
for i in range(len(matrix)):
    
    if ( (i == 0) | (i == 1) ):
        
        path = './kentucky/fairness/' + name[i] + '/two-year/misdemeanor/'
        ## confusion matrix and calibration only applies to cart, ebm, riskslim and stumps
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        
    else:
        path = './kentucky/fairness/' + name[i] + '/two-year/misdemeanor/'
        
        ## including arnold and compas now
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        arnold_matrix = arnold[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        arnold_matrix.to_csv(path+'arnold-'+name[i]+'.csv', index=None,header=True)